# Test on Guru Dataset
How robust are all previously developped models, regarding the selfmade guru dataset?

**Conclusion:** Performance recieves a shock.

In [1]:
# Compute on CPU (in case GPU is not available)
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import keras
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import kapre
from utils import DataFeed
import sys
sys.path.append('../preprocessing')

Using TensorFlow backend.


 ## Create Guru Dataset from raw data

In [3]:
from helper_functions import *
import os

In [4]:
raw_data_path ='../data/raw/guru'
data_path = '../preprocessing/preprocessed_data'

In [5]:
for lang in ['de', 'fr', 'en']:
    # For all files in raw_data_path
    for path in get_paths(os.path.join(raw_data_path, lang)):
        # open file with librosa
        part, sr = librosa.load(path, sr=16000, duration=5.0)
        # save as numpy array
        np.save(os.path.join(data_path, 'test/guru', lang, os.path.basename(path)), part)

## Load Data

In [6]:
data, labels = DataFeed.Dataset.create(data_path, ['test/guru'], num=-1, shuffle=True)
data.shape

(70, 1, 80000)

In [7]:
# Listen to a random sample
import IPython.display as ipd
ipd.Audio(data[-2], rate=16000)

## Evaluate

In [10]:
# Models to evaluate
models = ['conv2d_to_rnn/conv2d_to_rnn.h5', 'berlin_net/berlin_net.h5', 'conv2d_deep/conv2d_deep.h5', 'model_ens.h5']

In [12]:
# For all models
for mpath in models:
    # Load Model with keras
    model = load_model(mpath, custom_objects={'Melspectrogram':kapre.time_frequency.Melspectrogram, 
                                              'Normalization2D': kapre.utils.Normalization2D})
    model.compile(optimizer='RMSprop',
                  metrics=['accuracy', 'mae'],
                  loss='categorical_crossentropy')
    
    # Evaluate on test data
    results = model.evaluate(x=data, y=labels, verbose=0)
    # Print result
    print(f"{mpath}: {model.metrics_names[0]} = {results[0]},  {model.metrics_names[1]} = {results[1]}")

int_axis=0 passed but is ignored, str_axis is used instead.
conv2d_to_rnn/conv2d_to_rnn.h5: loss = 0.6193414160183498,  acc = 0.8571428554398673
int_axis=0 passed but is ignored, str_axis is used instead.
berlin_net/berlin_net.h5: loss = 0.6769303594316755,  acc = 0.8285714268684388
conv2d_deep/conv2d_deep.h5: loss = 1.8504622663770403,  acc = 0.5714285722800664
int_axis=0 passed but is ignored, str_axis is used instead.
int_axis=0 passed but is ignored, str_axis is used instead.
model_ens.h5: loss = 0.5421243378094265,  acc = 0.8285714268684388
